In [1]:
import pandas as pd
import folium
from folium.plugins import MarkerCluster

# Mini-Modeling Problem Solution

This is an example solution to the mini-modeling problem by Aidan Kingsbury.

1. Read in IDP data from the file "idp.xlsx"

In [2]:
#original filename = 'dtm-ethiopia-tigray-region-site-assessment-round-33-april-june-2023.xlsx'
filename="idp.xlsx"
df = pd.read_excel(filename)
#import the data
df.head(1)

,1.1.a.1: Survey Date,Country,Country Code,Reported Date,1.1.a.2: Survey Round,1.1.c.1: Site ID,1.1.d.1: Site Name,1.1.d.2: Site Alternate Name,1.4.a.2: Is site open?,1.1.e.1: Region,...,S1723: Other internet sources (e.g. apps),S1723: Please specify which other internet sources,S1723: Other,"S1723: If other source of news/information, please specify",S1784: Is mobile network access available in the site?,S1495: What % of HHs own a mobile phone?,11.3.a.1: Are members of the community discussing/advertising travel opportunities?,"11.3.a.6: If Yes, to where?",11.3.a.2: Specify all locations,M1712: Additional Comments / Observations
0,#date+occurred,#country+name,#country+code,#date+reported,NaN,NaN,NaN,NaN,NaN,#adm1+name,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


2. Select columns to assess education disorder.

In [3]:
#Pre-primary, secondary, resources,
#quality/satisfaction, girl attendance, boy attendance, teachers
columns_to_keep = ['1.1.c.1: Site ID',
                   '1.1.d.1: Site Name',
                   '1.1.f.1: GPS: Longitude',
                   '1.1.f.2: GPS: Latitude',
                   'S1518: Is there access to pre-primary (formal or temporary) schools for children from displaced HHs?',
                   'S1536: Access to secondary (formal or temporary) schools for children from displaced HHs?',
                   'S1226: Other, Specify',
                   'S1613: Poor educational standard',
                   'S1613: Why are some girls not attending school?',
                   'S1614: Why are some boys not attending school?',
                   'Are there enough teachers to school children?']
data = df[columns_to_keep]
new_column_names = ['ID', 'Name','Long','Lat','Pre-primary','Secondary','Resources','Quality','Girl_Attendence','Boy_Attendence','Teachers']
data.columns = new_column_names
data=data.dropna()

In [4]:
data.head(1)

,ID,Name,Long,Lat,Pre-primary,Secondary,Resources,Quality,Girl_Attendence,Boy_Attendence,Teachers
1,TG1039,Hareze Seb'ata,39.5828,14.43946,No,No,Education is not yet started,No,Other,Other,Unknown


3. Map IDP camps (optional)

In [5]:

# Assuming 'data' is your DataFrame with 'Lat' and 'Long' columns

# Create a folium map centered around the mean of latitudes and longitudes
map_center = [data['Lat'].mean(), data['Long'].mean()]
my_map = folium.Map(location=map_center, zoom_start=12)

# Create a MarkerCluster layer for better visualization of markers
marker_cluster = MarkerCluster().add_to(my_map)

# Add markers for each row in the DataFrame
for index, row in data.iterrows():
    folium.Marker([row['Lat'], row['Long']], popup=row['Name']).add_to(marker_cluster)

# Save the map as an HTML file or display it
my_map.save("my_map.html")
# or
my_map


4. Determine camps for Mekelle and Shire.

In [6]:
#Mekele = 13.4963° N, 39.4743°
#Shire = 14.1016° N, 38.2836° 
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the Haversine distance between two sets of latitude and longitude coordinates.
    """
    R = 3959.87433  # radius of the Earth in miles
    dlat = np.radians(lat2 - lat1)
    dlon = np.radians(lon2 - lon1)
    a = np.sin(dlat / 2)**2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    distance = R * c
    return distance

# Assuming 'data' is your DataFrame with 'Lat' and 'Long' columns
Mekele_coords = (13.4963, 39.4743)  # Replace with the coordinates of the first city
Shire_coords = (14.1016, 38.2836)  # Replace with the coordinates of the second city

# Function to determine the closest city based on Haversine distance
def closest_city(row):
    distance_to_Mekele = haversine(row['Lat'], row['Long'], *Mekele_coords)
    distance_to_Shire = haversine(row['Lat'], row['Long'], *Shire_coords)
    
    if distance_to_Mekele < 15 and distance_to_Mekele <= distance_to_Shire:
        return 'Mekele'
    elif distance_to_Shire < 15:
        return 'Shire'
    else:
        return 'Other'

# Add 'City' column to the DataFrame
data['City'] = data.apply(closest_city, axis=1)

In [7]:
data.head(1)

,ID,Name,Long,Lat,Pre-primary,Secondary,Resources,Quality,Girl_Attendence,Boy_Attendence,Teachers,City
1,TG1039,Hareze Seb'ata,39.5828,14.43946,No,No,Education is not yet started,No,Other,Other,Unknown,Other


5. Binary code education needs.

In [8]:
#Pre-primary: No = 0, Yes = 1
#Secondary: No = 0, Yes = 1
#Resources: 'Lack of scholastic material'=0, else 1
#Quality: No=0, Yes = 1
#Girl_Attendence: 'They are attending' = 1, else = 0
#Boy_Attendence: 'They are attending' = 1, else = 0
#Teachers: 'Yes' = 1, else = 0

# Assuming 'data' is your DataFrame with qualitative columns

# Define mapping functions for each column
def map_pre_primary(value):
    return 1 if 'Yes' in value else 0

def map_secondary(value):
    return 1 if 'Yes' in value else 0

def map_resources(value):
    return 0 if 'Lack of scholastic material' in value else 1

def map_quality(value):
    return 1 if 'Yes' in value else 0

def map_girl_attendance(value):
    return 1 if 'They are attending' in value else 0

def map_boy_attendance(value):
    return 1 if 'They are attending' in value else 0

def map_teachers(value):
    return 1 if 'Yes' in value else 0

# Apply the mapping functions to each column
data['Pre-primary'] = data['Pre-primary'].apply(map_pre_primary)
data['Secondary'] = data['Secondary'].apply(map_secondary)
data['Resources'] = data['Resources'].apply(map_resources)
data['Quality'] = data['Quality'].apply(map_quality)
data['Girl_Attendence'] = data['Girl_Attendence'].apply(map_girl_attendance)
data['Boy_Attendence'] = data['Boy_Attendence'].apply(map_boy_attendance)
data['Teachers'] = data['Teachers'].apply(map_teachers)

# Display the updated DataFrame
data.head()



,ID,Name,Long,Lat,Pre-primary,Secondary,Resources,Quality,Girl_Attendence,Boy_Attendence,Teachers,City
1,TG1039,Hareze Seb'ata,39.58280,14.43946,0,0,1,0,0,0,0,Other
2,TG1040,Enda Mosa,39.55493,14.42384,0,0,1,0,0,0,0,Other
3,TG729,Adi Abagie,38.18660,14.26590,0,0,1,0,0,0,0,Shire
4,TG732,May Ambssa,38.23350,14.22740,0,0,1,0,0,0,0,Shire
5,TG731,Hibret,38.16930,14.32020,0,0,1,0,0,0,0,Other


In [9]:

# Filter rows with 'Mekele' in the 'City' column
mekele_df = data[data['City'] == 'Mekele']

# Filter rows with 'Shire' in the 'City' column
shire_df = data[data['City'] == 'Shire']

In [10]:
mekele_df.head()

,ID,Name,Long,Lat,Pre-primary,Secondary,Resources,Quality,Girl_Attendence,Boy_Attendence,Teachers,City
468,TG624,Felege Mayat,39.4715,13.6018,1,0,0,0,1,1,0,Mekele
469,TG765,Amanit,39.2707,13.5746,1,0,0,0,0,1,0,Mekele
470,TG769,Metkel,39.3458,13.3702,1,0,0,0,1,1,0,Mekele
471,TG768,Dejen,39.3670,13.3412,1,0,0,0,1,1,1,Mekele
481,TG621,Derga Ajen,39.6829,13.5092,1,0,1,0,0,0,0,Mekele


In [11]:
shire_df.head()

,ID,Name,Long,Lat,Pre-primary,Secondary,Resources,Quality,Girl_Attendence,Boy_Attendence,Teachers,City
3,TG729,Adi Abagie,38.186600,14.265900,0,0,1,0,0,0,0,Shire
4,TG732,May Ambssa,38.233500,14.227400,0,0,1,0,0,0,0,Shire
6,TG1042,Tsaeda Ambora,38.128251,14.238904,0,0,1,0,0,0,0,Shire
36,TG730,Hadegiti,38.214900,14.267000,0,0,1,0,0,0,0,Shire
59,TG518,Adikokob,38.238100,14.212100,0,0,0,0,0,0,1,Shire


6. Compute entropy for IDP education data for Mekelle and Shire

In [12]:
import pandas as pd
import numpy as np

def calculate_entropy(df):
    entropy = 0
    total_rows = len(df)

    # Calculate the probability of each need
    for column in ['Pre-primary', 'Secondary', 'Resources', 'Quality', 'Girl_Attendence', 'Boy_Attendence', 'Teachers']:
        # Count the occurrences of 1 and 0 in the column
        count_1 = df[column].sum()
        count_0 = total_rows - count_1

        # Calculate the probability
        p_1 = count_1 / total_rows
        p_0 = count_0 / total_rows

        # Avoid log(0) by adding a small value (epsilon)
        epsilon = 1e-10

        # Update the entropy using the entropy formula
        entropy -= (p_1 * np.log2(p_1 + epsilon) + p_0 * np.log2(p_0 + epsilon))

    return entropy

# Calculate entropy for 'Mekele' DataFrame
entropy_mekele = calculate_entropy(mekele_df)
print(f"Entropy for Mekele: {entropy_mekele}")

# Calculate entropy for 'Shire' DataFrame
entropy_shire = calculate_entropy(shire_df)
print(f"Entropy for Shire: {entropy_shire}")


Entropy for Mekele: 2.925327967431285
Entropy for Shire: 2.1418191760456615
